In [5]:
from dataloader import load_data
from learning_func import Ranger, scheduler
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Activation, AveragePooling2D, Input, Flatten, add
from tensorflow.keras.models import Sequential
from tensorflow import keras
#!pip install -U tensorflow-addons
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import EarlyStopping     


train_dir = "C:/Users/jonas/Desktop/HoferThomas/Dataset_Resnet_small"
val_dir = "C:/Users/jonas/Desktop/HoferThomas/Dataset_Resnet_small"

train_ds, val_ds, class_names = load_data(train_dir, val_dir)
print(class_names)
#model, history = train_model(train_ds, val_ds, class_names)

ImportError: cannot import name 'Ranger' from 'learning_func' (c:\Users\jonas\Desktop\HoferThomas\SJ_Resnet\learning_func.py)

In [ ]:
pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(270,480,3),
                   pooling='avg',classes=2,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model = Sequential()
resnet_model.add(pretrained_model)
resnet_model.add(Dense(1, activation='sigmoid'))

resnet_model.summary()


save_callback = EarlyStopping(
    monitor="val_accuracy",
    min_delta=0,
    patience=50,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)

lr_scheduler = keras.callbacks.LearningRateScheduler(scheduler, verbose=1)

resnet_model.compile(optimizer=Ranger(), loss='binary_crossentropy', metrics=['accuracy'])


epochs=50
history = resnet_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  shuffle=True,
  callbacks=[save_callback]
)